In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np
import os
import pandas as pd
import pathlib
import random

#tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# Im using the MNIST as jpeg dataset from kaggle 
# https://www.kaggle.com/scolianni/mnistasjpg

input_file_location = 'Path to dataset'

# Defining the function to return an appropriate input function for the estimator

In [13]:
def get_input_fn(mode, input_file_location, batch_size):
    
    def preprocess_image(image):
        image = tf.image.decode_jpeg(image, channels=1)
        image = tf.image.resize_images(image, [28, 28])
        image /= 255.0  # normalize to [0,1] range
        return image

    def load_and_preprocess_image(path):
        image = tf.read_file(path)
        return preprocess_image(image)

    def load_and_preprocess_from_path_label(path, label):
        return load_and_preprocess_image(path), tf.string_to_number(label, out_type=tf.int32)
    
    def _input_fn():
        
        data_root = pathlib.Path(input_file_location)
        
        ### Generating Lables from Directory Names
        label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
        
        label_to_index = dict((name, index) for index,name in enumerate(label_names))
        
        ### Getting Image paths
        all_image_paths = list(data_root.glob('*/*'))
        all_image_paths = [str(path) for path in all_image_paths if '.jpg' in str(path)]
        random.shuffle(all_image_paths)
        
        image_count = len(all_image_paths)
        
        all_image_labels = [pathlib.Path(path).parent.name for path in all_image_paths]
        
        
        full_dataset = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels)).map(load_and_preprocess_from_path_label)
        
        full_dataset = full_dataset.shuffle(4).repeat().batch(batch_size).prefetch(buffer_size=8)
        
        
        images, labels = full_dataset.make_one_shot_iterator().get_next()
        
        return images, labels
    
    return _input_fn

# Defining the convolutional neural network function

In [14]:
def cnn_model_fn(features, labels, mode, params):
    """Model function for CNN."""
    # Input Laye
    # Convolutional Layer #1
    layer_init = tf.layers.conv2d(inputs = features, filters=1,kernel_size=[1, 1],padding="same")
    input_layer = tf.reshape(layer_init, [-1, 28, 28, 1])
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=params.num_classes)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=params.learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Function for initializing the estimator

In [15]:
def create_estimator_and_specs(run_config):
    """Creates an Experiment configuration based on the estimator and input fn."""
    model_params = tf.contrib.training.HParams(
        batch_size=32,
        num_classes=10,
        learning_rate=0.01)

    estimator = tf.estimator.Estimator(
        model_fn=cnn_model_fn,
        config=run_config,
        params=model_params)

    train_spec = tf.estimator.TrainSpec(input_fn=get_input_fn(
        mode=tf.estimator.ModeKeys.TRAIN,
        input_file_location=input_file_location,
        batch_size=32), max_steps=5)

    eval_spec = tf.estimator.EvalSpec(input_fn=get_input_fn(
        mode=tf.estimator.ModeKeys.EVAL,
        input_file_location=input_file_location,
        batch_size=32))

    return estimator, train_spec, eval_spec

# Creating the estimator

In [16]:
estimator, train_spec, eval_spec = create_estimator_and_specs(
        run_config=tf.estimator.RunConfig(
        model_dir="cnn_outside",
        save_checkpoints_secs=300,
        save_summary_steps=100))

INFO:tensorflow:Using config: {'_model_dir': 'cnn_outside', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020339B47EB8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Starting the training

In [17]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn_outside\model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into cnn_outside\model.ckpt.
INFO:tensorflow:loss = 2.4104428, step = 1
INFO:tensorflow:Saving checkpoints for 5 into cnn_outside\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-07-15:29:19
INFO:tensorflow:Graph was finalized.


({'accuracy': 0.0925, 'global_step': 5, 'loss': 2.3041801}, [])